<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Content Based Personalization
## LightGBM on Azure Databricks<br>
This notebook provides a quick example of how to train LightGBM model on Azure Databricks and deploy it using MML Spark for a content personalization scenario.<br><br>
LightGBM \[1\] is a gradient boosting framework that uses tree-based learning algorithms.<br>
MML Spark \[2\] allows LightGBM to be called in a Spark environment which provides several advantages:
- Distributed computation for model development
- Easy integration into existing Spark workflows
- Model serving through Spark Serving \[3\]

## Global Settings and Imports

A python script is provided to simplify setting up Azure Databricks with the correct
dependencies.<br> Run ```python scripts/databricks_install.py -h``` for more details.

In [20]:
import os
import sys
from tempfile import TemporaryDirectory
sys.path.append("../../")

import pyspark
from pyspark.ml import PipelineModel
from pyspark.ml.feature import FeatureHasher
from pyspark.sql.functions import col, udf
from pyspark.sql.types import FloatType
import requests

from reco_utils.common.spark_utils import start_or_get_spark
from reco_utils.common.notebook_utils import is_databricks
from reco_utils.dataset.criteo import load_spark_df
from reco_utils.dataset.spark_splitters import spark_random_split

print("System version: {}".format(sys.version))
print("PySpark version: {}".format(pyspark.version.__version__))

System version: 3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]
PySpark version: 2.3.1


In [22]:
# Setup MML Spark
if not is_databricks():
    # get the maven coordinates for MML Spark from databricks_install script
    from scripts.databricks_install import MMLSPARK_INFO
    packages = [MMLSPARK_INFO['maven']['coordinates']]
    spark = start_or_get_spark(packages=packages)

from mmlspark import ComputeModelStatistics
from mmlspark import LightGBMClassifier

In [25]:
from azureml.core import Workspace
from azureml.core.model import Model

print("Azure ML SDK version: ", azureml.core.VERSION)

ModuleNotFoundError: No module named 'azure.mgmt.authorization'

## Data Preparation
The Criteo Display Advertising Challenge (DAC) dataset [4] is a well-known industry benchmarking dataset for developing CTR prediction models, and is used frequently by research papers. The original dataset contains over 45M rows, but there is also a down-sampled dataset which has 100,000 rows (this can be used by setting DATA_SIZE = 'sample').<br><br>
The dataset contains 1 label column and 38 feature columns, where 13 columns are integer values (int00-int12) and 25 columns are categorical features (cat00-cat24).<br><br>
What the columns represent is not provided, but for this case we can consider the integer and categorical values as features representing the user and / or item content. The label is binary and indicates a user interaction with an item, so this is a useful dataset to demonstrate how to build a model that will predict likelihood of a user interacting with an item based on the user and item content features.


In [4]:
DATA_SIZE = 'sample'

In [5]:
raw_data = load_spark_df(size=DATA_SIZE, spark=spark)

8.79MB [00:02, 3.62MB/s]                            


### Feature Processing
The feature data provided has many missing values across both integer and categorical feature fields. In addition the categorical features have many distinct values, so effectively cleaning and representing the feature data is an important step prior to training a model.<br><br>
One of the simplest ways of managing both features that have missing values as well as high cardinality is to use the hashing trick. The FeatureHasher transformer will pass integer values through and will hash categorical features into a sparse vector of lower dimensionality which can be used effectively by LightGBM.<br><br>
First the dataset is split randomly for training and testing and feature processing is applied to each dataset.

In [6]:
raw_train, raw_test = spark_random_split(raw_data, ratio=0.8, seed=42)

In [7]:
columns = [c for c in raw_data.columns if c != 'label']
feature_processor = FeatureHasher(inputCols=columns, outputCol='features')

In [8]:
train = feature_processor.transform(raw_train)
test = feature_processor.transform(raw_test)

In [9]:
# save data to tables to finalize feature transformation
train.write.mode('overwrite').saveAsTable('train')
test.write.mode('overwrite').saveAsTable('test')

In [10]:
# load data from table
train = spark.table('train')
test = spark.table('test')

## Model Training
In MML Spark the LightGBM implementation for binary classification is invoked using the LightGBMClassifier class and specifying the objective as 'binary'. In this instance the occurrence of positive labels is quite low, so setting the isUnbalance flag to true helps account for this imbalance.<br><br>

### Hyper-parameters
Below are some of the key hyper-parameters \[5\] for training a LightGBM classifier on Spark
- numLeaves: the number of leaves in each tree
- numIterations: the number of iterations to apply boosting
- learningRate: the learning rate for training across trees
- featureFraction: the fraction of features used for training a tree
- earlyStoppingRound: round at which early stopping can be applied to avoid overfitting

In [11]:
NUM_LEAVES = 32
NUM_ITERATIONS = 10
LEARNING_RATE = 0.15
FEATURE_FRACTION = 0.8
EARLY_STOPPING_ROUND = 20

In [12]:
lgbm = LightGBMClassifier(
    labelCol='label',
    featuresCol='features',
    objective='binary',
    isUnbalance=True,
    boostingType='gbdt',
    boostFromAverage=True,
    numLeaves=NUM_LEAVES,
    numIterations=NUM_ITERATIONS,
    learningRate=LEARNING_RATE,
    featureFraction=FEATURE_FRACTION,
    earlyStoppingRound=EARLY_STOPPING_ROUND,
)

### Model Training and Evaluation

In [ ]:
model = lgbm.fit(train)
predictions = model.transform(test)

In [16]:
evaluator = (
    ComputeModelStatistics()
    .setScoredLabelsCol("prediction")
    .setLabelCol("label")
    .setEvaluationMetric("AUC")
)

evaluator.transform(predictions).show()

+---------------+------------------+
|evaluation_type|               AUC|
+---------------+------------------+
| Classification|0.6310011615829604|
+---------------+------------------+



## Model Saving and Loading
The full pipeline for operating on raw data including feature processing and model prediction can be saved and reloaded for use in another workflow.

In [34]:
tmp = TemporaryDirectory()

# save model to temporary directory
model_name = 'finished.model'
save_file = os.path.join(tmp.name, model_name)
pipeline = PipelineModel(stages=[feature_processor, model])
pipeline.save(save_file)

# zip file for transfer to Azure ML Workspace
zip_file = shutil.make_archive(base_name=model_name, 
                               format='zip', 
                               root_dir='/dbfs{}'.format(save_file))

# register model in Azure ML Workspace
ws = Workspace.from_config()
model = Model.register(workspace=ws, model_path=zip_file, model_name=model_name)

In [ ]:
# cleanup temporary directory
tmp.cleanup()

## Reference
\[1\] Guolin Ke, Qi Meng, Thomas Finley, Taifeng Wang, Wei Chen, Weidong Ma, Qiwei Ye, and Tie-Yan Liu. 2017. LightGBM: A highly efficient gradient boosting decision tree. In Advances in Neural Information Processing Systems. 3146–3154.<br>
\[2\] MML Spark: https://mmlspark.blob.core.windows.net/website/index.html <br>
\[3\] MML Spark Serving: https://github.com/Azure/mmlspark/blob/master/docs/mmlspark-serving.md <br>
\[4\] The Criteo dataset: http://labs.criteo.com/wp-content/uploads/2015/04/dac_sample.tar.gz <br>
